In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-06 01:31:12,115 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 01:31:12,129 INFO: Initializing external client
2025-03-06 01:31:12,130 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:31:13,262 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Fetching data from 2025-02-05 06:31:12.115737+00:00 to 2025-03-06 05:31:12.115737+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.36s) 


In [5]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 01:31:29,981 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 01:31:30,002 INFO: Initializing external client
2025-03-06 01:31:30,002 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:31:30,949 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672

In [6]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [7]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,255,1.0,2025-03-06 07:00:00+00:00
1,218,1.0,2025-03-06 07:00:00+00:00
2,237,126.0,2025-03-06 07:00:00+00:00
3,57,0.0,2025-03-06 07:00:00+00:00
4,56,0.0,2025-03-06 07:00:00+00:00
...,...,...,...
247,71,1.0,2025-03-06 07:00:00+00:00
248,8,0.0,2025-03-06 07:00:00+00:00
249,168,2.0,2025-03-06 07:00:00+00:00
250,186,91.0,2025-03-06 07:00:00+00:00


In [8]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-06 01:31:48,461 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 01:31:48,475 INFO: Initializing external client
2025-03-06 01:31:48,475 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:31:49,376 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 252/252 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215672/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)